# **Inference pipeline**
___  
  
In this notebook we show how to apply **inference** with [BiaPy](https://biapy.readthedocs.io/en/latest/).  

**Without any coding**, we explain step by step how to
1. **upload a set of test images** with their corresponding labels,
2. **apply the model** to the test images, and
4. **download the results** to your local machine.

**Disclaimer:** the structure of the notebook is heavily inspired in the fantastic [ZeroCostDL4Mic notebooks](https://github.com/HenriquesLab/ZeroCostDL4Mic/wiki).

**Contact:** This notebook has been made by [Ignacio Arganda-Carreras](mailto:ignacio.arganda@ehu.eus) and [Daniel Franco-Barranco](mailto:daniel.franco@dipc.org). If you have any suggestion or comment, or find any problem, please write us an email or [create an issue in BiaPy's repository](https://github.com/BiaPyX/BiaPy/issues). Thanks!

## **Expected inputs and outputs**
___
**Inputs**

This notebook expects three folders as input:
* **Test raw images**: with the raw images to test the model.
* **Test labels**: ground truth of the test images. Depending on the workflow to be used, the inputs will be images or CSV files.
* **Output folder**: a path to store the results.

**Outputs**

Depending on the workflow, the output could be an image, or a csv file per each test sample.

<font color='red'><b>Note</b></font>: for testing purposes, you can also run this notebook with the **example datasets provided in 'Manage file(s) source > Option 3'**.

**Data structure**

To ensure the proper operation of the library the data directory tree should be something like this:

```
dataset/
└── test
    ├── x
    │   ├── testing-0001.tif
    │   ├── testing-0002.tif
    │   ├── . . .
    │   ├── testing-9999.tif
    └── y
        ├── testing_groundtruth-0001.tif
        ├── testing_groundtruth-0002.tif
        ├── . . .
        ├── testing_groundtruth-9999.tif
```

**⚠️ Warning:** Ensure that images and their corresponding ground truth files are sorted in the same way. A common approach is to fill with zeros the image number added to the filenames (as in the example).

**Input Format Support**

This notebook is compatible with a range of input formats. You can use the following file extensions: `.tif`, `.png`, `.jpg`, `.npy`, `.h5`, `.hdf5` (every extension supported by [scikit-image](https://scikit-image.org/docs/stable/api/skimage.io.html#skimage.io.imread)).





## **Prepare the environment**
___

Establish connection with Google services. You **must be logged in to Google** to continue.
Since this is not Google's own code, you will probably see a message warning you of the dangers of running unfamiliar code. This is completely normal.



## **Check for GPU access**
---

By default, the session should be using Python 3 and GPU acceleration, but it is possible to ensure that these are set properly by doing the following:

Go to **Runtime -> Change the Runtime type**

**Runtime type: Python 3** *(Python 3 is programming language in which this program is written)*

**Accelerator: GPU** *(Graphics processing unit)*

## **Manage file(s) source**
---
The input folder can be provided using three different options: by directly uploading the folder (option 1), by using a folder stored in Google Drive (option 2) or by using a few samples of our data (option 3).

Depending on the option chosen, different steps will have to be taken, as explained in the following cells.


### **Option 1: Upload Files from Your Local Machine**
---
You will be prompted to upload your files to Colab and they will be stored under `/content/input/`.

In [ ]:
#@markdown ##Play the cell to upload local files (test raw images)

from google.colab import files
!mkdir -p /content/input/test/x
%cd /content/input/test/x
uploaded = files.upload()
%cd /content

In [ ]:
#@markdown ##Play the cell to upload local files (test labels)

from google.colab import files
!mkdir -p /content/input/test/y
%cd /content/input/test/y
uploaded = files.upload()
%cd /content

In [ ]:
# @title  { display-mode: "form" }
#@markdown ##Play the cell to upload local files (YAML file)

from google.colab import files
%cd /content
uploaded = files.upload()


In [ ]:
# @title  { display-mode: "form" }
#@markdown ##Play the cell to upload local files (model's weights)

from google.colab import files
%cd /content
uploaded = files.upload()


### **Option 2: Mount Your Google Drive**
---
To use this notebook on your own data from Google Drive, you need to mount Google Drive first.

Play the cell below to mount your Google Drive and follow the link that will be shown. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive.

Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab

#@markdown * Click on the URL.

#@markdown * Sign in your Google Account.

#@markdown * Copy the authorization code.

#@markdown * Enter the authorization code.

#@markdown * Click on "Files" site on the right. Refresh the site. Your Google Drive folder should now be available here as "drive".

# mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/gdrive')

### **Option 3: Download an Example Dataset**
---
If you do not have data, YAML file and model weights at hand but would like to test the notebook, no worries! You can run the following cells to download our example data for 2D semantic segmentation. We will use the [Electron Microscopy Dataset (EPFL - CVLAB)](https://www.epfl.ch/labs/cvlab/data/data-em/) publicly available online.

In [ ]:
#@markdown ##Play to download an example dataset
import os

!pip install gdown==4.7.3 --quiet
import gdown

os.chdir('/content/')
gdown.download("https://drive.google.com/uc?id=1DfUoVHf__xk-s4BWSKbkfKYMnES-9RJt", "fibsem_epfl.zip", quiet=True)

!unzip -q fibsem_epfl.zip
!rm fibsem_epfl.zip
print( 'Dataset downloaded and unzipped under /content/data')


Dataset downloaded and unzipped under /content/data


In [ ]:
#@markdown ##Play to download an example YAML file

import os

os.chdir('/content/')

!pip install --upgrade --no-cache-dir gdown &> /dev/null
!gdown --id 12rdUJnsmOkkP4r4LzyAkFPMVUSN-lfni &> /dev/null
print('YAML file downloaded under /content/')


YAML file downloaded under /content/


In [ ]:
#@markdown ##Play to download model's weights

import os

os.chdir('/content/')

!pip install --upgrade --no-cache-dir gdown &> /dev/null
!gdown --id 1UdPq_3bp1SvO7SBJz1nBI9mmri6m0oEP &> /dev/null
print( "Model's weigths downloaded under /content/data")


Model's weigths downloaded under /content/data


## **Paths for Input Images and Output Files**
___

Depending on the option you chose for managing file sources, you'll set your paths differently:

- **Option 1 (Upload from Local Machine)**:
  - Set `test_data_path` to `/content/input/test/x`
  - Set `test_data_gt_path` to `/content/input/test/y`
  - Set `yaml_file` to `/content/your_yaml_file.yaml`
  - Set `checkpoint_file` to `/content/your_checkpoint.pth`
  - Set `output_path` to `/content/out`
  
- **Option 2 (Use Google Drive Data)**:
  - Insert the paths to your input files and your desired output directory here, i.e., `/content/gdrive/MyDrive/...`.
  
- **Option 3 (Use Our Sample Data)**:
  - Set `test_data_path` to `/content/data/test/x`
  - Set `test_data_gt_path` to `/content/data/test/y`
  - Set `yaml_file` to `/content/your_yaml_file.yaml`
  - Set `checkpoint_file` to `/content/your_checkpoint.pth`
  - Set `output_path` to `/content/out`
  
  **Note**: Ensure you download your results from the `/content/out` directory after the process!

**Helpful Tip**: If you're unsure about the paths to your folders, look at the top left of this notebook for a small folder icon. Navigate through the directories until you locate your desired folder. Right-click on it and select "Copy Path" to copy the folder's path.

In [ ]:
#@markdown #####Path to test images
test_data_path = '/content/data/test/x' #@param {type:"string"}
#@markdown #####Path to test ground truth (if exists)
test_data_gt_path = '/content/data/test/y' #@param {type:"string"}
#@markdown #####Path to the YAML configuration file
yaml_file = '/content/p_semantic.yaml' #@param {type:"string"}
#@markdown #####Path to checkpoint file
checkpoint_file = '/content/model_weights_p_semantic_1.pth' #@param {type:"string"}
#@markdown #####Path to store the resulting images (it'll be created if not existing):
output_path = '/content/output' #@param {type:"string"}

## **Install BiaPy library**


In [ ]:
#@markdown ##Play to install BiaPy and its dependences

import os
import sys
import numpy as np
from tqdm.notebook import tqdm
from skimage.io import imread
from skimage.exposure import match_histograms

!pip install biapy==3.3.2

from biapy import BiaPy

Cloning into 'BiaPy'...
remote: Enumerating objects: 15964, done.
remote: Counting objects: 100% (2033/2033), done.
remote: Compressing objects: 100% (609/609), done.
remote: Total 15964 (delta 1442), reused 1991 (delta 1404), pack-reused 13931
Receiving objects: 100% (15964/15964), 827.17 MiB | 14.79 MiB/s, done.
Resolving deltas: 100% (8711/8711), done.


### **Select your parameters**
---
* **`load_gt`:** Select to load ground truth labels and measure output performance. **Default value: True**

In [ ]:
#@markdown ###Name of the model:
load_gt = True #@param {type:"boolean"}

In [ ]:
#@markdown ##Play to ensure only inference will be done
import errno

os.chdir('/content/')

# Check folders before modifying the .yaml file
if not os.path.exists(test_data_path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), test_data_path)
ids = sorted(next(os.walk(test_data_path))[2])
if len(ids) == 0:
    raise ValueError("No images found in dir {}".format(test_data_path))

if not os.path.exists(yaml_file):
    raise ValueError("No YAML configuration file found in {}".format(yaml_file))

if not os.path.exists(checkpoint_file):
    raise ValueError("No h5 checkpoint file found in {}".format(checkpoint_file))


# open template configuration file
import yaml
with open( yaml_file, 'r') as stream:
    try:
        biapy_config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

biapy_config['DATA']['TEST']['PATH'] = test_data_path
biapy_config['DATA']['TEST']['GT_PATH'] = test_data_gt_path

biapy_config['DATA']['TEST']['LOAD_GT'] = load_gt
biapy_config['TRAIN']['ENABLE'] = False
biapy_config['TEST']['ENABLE'] = True
biapy_config['MODEL']['LOAD_CHECKPOINT']= True
biapy_config['PATHS'] = {}
biapy_config['PATHS']['CHECKPOINT_FILE']= checkpoint_file

# save file
with open( yaml_file, 'w') as outfile:
    yaml.dump(biapy_config, outfile, default_flow_style=False)

print( "Inference configuration finished.")

Inference configuration finished.


### **Make the inference**
---

In [ ]:
#@markdown ##Play to train the model

import os
import errno
job_name = os.path.splitext(yaml_file)[0].split('/')[-1]

# Run the code
biapy = BiaPy(yaml_file, result_dir=output_path, name=job_name, run_id=1, gpu=0)
biapy.run_job()


2023-02-21 12:51:41.131821: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 12:51:42.425094: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-21 12:51:42.425245: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-21 12:51:42.425266: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

## **Download results**

In [ ]:
#@markdown ###Play to download a zip file with all the results in test.

from google.colab import files

os.chdir('/content/')

!zip -r -q /content/output.zip $output_path

files.download("/content/output.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Acknowledgments**
We would like to acknowledge the inspiration provided by the excellent [ZeroCostDL4Mic notebooks](https://github.com/HenriquesLab/ZeroCostDL4Mic/wiki). In particular, we have reused some of their descriptions of metrics and parameters, as well as their 3D visualization widget code from their [U-Net 3D notebook](https://colab.research.google.com/github/HenriquesLab/ZeroCostDL4Mic/blob/master/Colab_notebooks/U-Net_3D_ZeroCostDL4Mic.ipynb).